In [10]:
import pandas as pd
import numpy as np

In [11]:
data = pd.read_csv('german.data-numeric', delim_whitespace=True, header=None)

In [12]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1,6,4,12,5,5,3,4,1,67,...,0,0,1,0,0,1,0,0,1,1
1,2,48,2,60,1,3,2,2,1,22,...,0,0,1,0,0,1,0,0,1,2
2,4,12,4,21,1,4,3,3,1,49,...,0,0,1,0,0,1,0,1,0,1
3,1,42,2,79,1,4,3,4,2,45,...,0,0,0,0,0,0,0,0,1,1
4,1,24,3,49,1,3,3,4,4,53,...,1,0,1,0,0,0,0,0,1,2


In [13]:
column_names = [
    'Status_checking_account', 'Duration', 'Credit_history', 'Purpose',
    'Credit_amount', 'Savings_account', 'Present_employment', 'Installment_rate',
    'Personal_status_sex', 'Other_debtors', 'Present_residence', 'Property',
    'Age', 'Other_installment_plans', 'Housing', 'Number_credits', 'Job',
    'Number_people_liable', 'Telephone', 'Foreign_worker','1','2','3','4', 'Target'
]

In [14]:
data.columns = column_names

In [15]:
X = data.drop(['Target','1','2','3','4'], axis=1)
y = data['Target']

In [16]:
X.head()

,Status_checking_account,Duration,Credit_history,Purpose,Credit_amount,Savings_account,Present_employment,Installment_rate,Personal_status_sex,Other_debtors,Present_residence,Property,Age,Other_installment_plans,Housing,Number_credits,Job,Number_people_liable,Telephone,Foreign_worker
0,1,6,4,12,5,5,3,4,1,67,3,2,1,2,1,0,0,1,0,0
1,2,48,2,60,1,3,2,2,1,22,3,1,1,1,1,0,0,1,0,0
2,4,12,4,21,1,4,3,3,1,49,3,1,2,1,1,0,0,1,0,0
3,1,42,2,79,1,4,3,4,2,45,3,1,2,1,1,0,0,0,0,0
4,1,24,3,49,1,3,3,4,4,53,3,2,2,1,1,1,0,1,0,0


In [17]:
y.head()

0    1
1    2
2    1
3    1
4    2
Name: Target, dtype: int64

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
def cost_sensitive_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    cost = cm[0, 1] * 1 + cm[1, 0] * 5  # Cost matrix implementation
    return -cost  # Negative cost as we want to minimize it

cost_scorer = make_scorer(cost_sensitive_score, greater_is_better=False)

# Create and tune the Random Forest classifier
rfc = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [20]:
grid_search = GridSearchCV(estimator=rfc, param_grid=param_grid, scoring=cost_scorer, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Predict and evaluate on the test set
y_pred = best_model.predict(X_test)

# Confusion matrix and cost calculation
cm = confusion_matrix(y_test, y_pred)
cost = cm[0, 1] * 1 + cm[1, 0] * 5

print(f"Confusion Matrix:\n{cm}")
print(f"Total Cost: {cost}")

Confusion Matrix:
[[196  13]
 [ 60  31]]
Total Cost: 313
